In [17]:
import cv2
import mediapipe as mp
import time


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Define a function to detect sign based on finger positions
def detect_sign(landmarks):
    
    tip_ids = [4, 8, 12, 16, 20]  
    fingers_up = []

# Thumb (check x-coordinate for left/right hand detection)
    if landmarks[tip_ids[0]].x < landmarks[tip_ids[0] - 1].x:
        fingers_up.append(1)
    else:
        fingers_up.append(0)

# Other fingers
    for id in range(1, 5):
        if landmarks[tip_ids[id]].y < landmarks[tip_ids[id] - 2].y:
            fingers_up.append(1)
        else:
            fingers_up.append(0)

# Simple example rules
    if fingers_up == [0, 1, 0, 0, 0]:
        return "Pointing"
    elif fingers_up == [0, 1, 1, 0, 0]:
        return "Peace"
    elif fingers_up == [0, 0, 1, 1, 1]:
        return "Excellent"
    elif fingers_up == [1, 0, 0, 0, 0]:
        return "Thumbs up"
    elif fingers_up == [1, 1, 1, 1, 1]:
        return "Open Palm"
    elif fingers_up == [0, 0, 0, 0, 0]:
        return "Fist"
    else:
        return "Unknown"

cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    prev_time = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame for natural viewing
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        sign_name = "Unknown"

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                sign_name = detect_sign(hand_landmarks.landmark)
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Get frame size
        height, width, _ = frame.shape

        # Draw sign name at bottom center in BLACK 
        cv2.putText(frame, sign_name, (width // 2 - 50, height - 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3, cv2.LINE_AA)

        cv2.imshow("Finger Sign Detection", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC key to exit
            break

cap.release()
cv2.destroyAllWindows()
